In [124]:
import math
import numpy as np

from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Reshape, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

from gpaulo.game import suggest
from gpaulo.train import mutate_data

In [132]:
# load data
data = np.load('data/tictactoe.npz')['data']

# strategy data
favs = np.array([
    # center start
    [[0,0,0, 0,-1,0, 0,0,0], [0,0,0, 0,0,0, 0,0,1]],
    [[0,0,-1, 0,1,0, 0,0,0], [1,0,0, 0,0,0, 0,0,1]],
    [[1,0,-1, 0,1,0, 0,0,-1], [0,0,0, 0,0,1, 0,0,0]],
    [[1,0,-1, -1,1,1, 0,0,-1], [0,0,0, 0,0,0, 0,1,0]],

    # my strategy (classic)
    [[0,0,0, 0,-1,0, 1,0,0], [0,0,1, 0,0,0, 0,0,0]],
    [[0,0,1, 0,-1,0, 1,0,-1], [1,0,0, 0,0,0, 0,0,0]],
    [[-1,0,1, 0,-1,0, 1,0,0], [0,0,0, 0,0,0, 0,0,1]],

    [[1,0,1, -1,-1,0, 1,0,-1], [0,1,0, 0,0,0, 0,0,0]],
    [[1,-1,1, 0,-1,0, 1,0,-1], [0,0,0, 1,0,0, 0,0,0]],
    [[-1,0,1, 0,-1,-1, 1,0,1], [0,0,0, 0,0,0, 0,1,0]],
    [[-1,0,1, 0,-1,0, 1,-1,1], [0,0,0, 0,0,1, 0,0,0]],

    # my strategy (reverse corner)
    [[0,0,-1, 0,0,0, 1,0,0], [1,0,0, 0,0,0, 0,0,1]],
    [[1,0,-1, -1,0,0, 1,0,0], [0,0,0, 0,0,0, 0,0,1]],
    [[0,0,-1, 0,0,0, 1,-1,1], [1,0,0, 0,0,0, 0,0,0]],

    [[1,0,-1, -1,0,0, 1,-1,1], [0,0,0, 1,1,0, 0,0,0]],
    [[1,0,-1, 0,-1,0, 1,-1,1], [0,0,0, 1,1,0, 0,0,0]],
    [[1,0,-1, -1,0,0, 1,-1,1], [0,0,0, 0,1,0, 0,1,0]],
    [[1,0,-1, -1,-1,0, 1,0,1], [0,0,0, 0,1,0, 0,1,0]],

    # my strategy (another corners)
    [[0,0,0, 0,0,0, 1,0,-1], [0,0,1, 0,0,0, 0,0,0]],
    [[-1,0,0, 0,0,0, 1,0,0], [0,0,1, 0,0,0, 0,0,0]],
    [[-1,0,1, 0,-1,0, 1,0,0], [0,0,0, 0,0,0, 0,0,1]],
    [[0,0,1, 0,-1,0, 1,0,-1], [1,0,0, 0,0,0, 0,0,0]],

    [[1,-1,1, 0,-1,0, 1,0,-1], [0,0,0, 1,0,0, 0,0,0]],
    [[1,0,1, -1,-1,0, 1,0,-1], [0,1,0, 0,0,0, 0,0,0]],
    [[-1,0,1, 0,-1,-1, 1,0,1], [0,0,0, 0,0,0, 0,1,0]],
    [[-1,0,1, 0,-1,0, 1,-1,1], [0,0,0, 0,0,1, 0,0,0]],

    [[-1,-1,1, 0,0,0, 1,0,0], [0,0,0, 0,1,0, 0,0,0]],
    [[-1,0,1, 0,0,0, 1,0,-1], [0,0,0, 0,1,0, 0,0,0]],
    
    # my strategy (another cells)
    [[0,0,0, 0,0,0, 1,-1,0], [0,0,0, 0,1,0, 0,0,0]],
    [[0,0,0, 0,0,-1, 1,0,0], [0,0,0, 0,1,0, 0,0,0]],
    [[0,0,0, -1,0,0, 1,0,0], [0,0,0, 0,1,0, 0,0,0]],
    [[0,-1,0, 0,0,0, 1,0,0], [0,0,0, 0,1,0, 0,0,0]],

    [[0,0,-1, 0,1,0, 1,-1,0], [1,0,0, 0,0,0, 0,0,0]],
    [[0,0,-1, 0,1,-1, 1,0,0], [0,0,0, 0,0,0, 0,0,1]],
    [[0,0,-1, -1,1,0, 1,0,0], [0,0,0, 0,0,0, 0,0,1]],
    [[0,-1,-1, 0,1,0, 1,0,0], [1,0,0, 0,0,0, 0,0,0]],

    [[1,0,-1, -1,1,0, 1,-1,0], [0,0,0, 0,0,0, 0,0,1]],
    [[1,-1,-1, -1,1,0, 1,0,0], [0,0,0, 0,0,0, 0,0,1]],
    [[1,0,-1, 0,1,0, 1,-1,-1], [0,0,0, 1,0,0, 0,0,0]],
    [[1,-1,-1, 0,1,0, 1,0,-1], [0,0,0, 1,0,0, 0,0,0]],
    [[-1,0,-1, 0,1,-1, 1,0,1], [0,0,0, 0,0,0, 0,1,0]],
    [[-1,0,-1, -1,1,0, 1,0,1], [0,0,0, 0,0,0, 0,1,0]],
    [[0,0,-1, 0,1,-1, 1,-1,1], [1,0,0, 0,0,0, 0,0,0]],
    [[0,0,-1, -1,1,0, 1,-1,1], [1,0,0, 0,0,0, 0,0,0]],
])
# favs = mutate_data(favs)
favs = np.concatenate([favs] * 150)

# fixes
fixes = np.array([
    [[1,0,0,-1,1,0,-1,0,-1], [0,0,0,0,0,0,0,1,0]],
    [[0,0,0,-1,-1,0,0,0,1], [0,0,0,0,0,1,0,0,0]],
    [[1,-1,-1, 0,1,0, 0,0,-1], [0,0,0,0,0,1,0,0,0]],
    [[0,0,0, -1,1,0, -1,1,0], [1,0,0, 0,0,0, 0,0,0]],
    [[0,-1,-1, 0,1,0, 1,0,0], [1,0,0, 0,0,0, 0,0,0]],
    [[0,0,1, 0,1,-1, -1,0,-1], [0,0,0, 0,0,0, 0,1,0]],
    [[0,0,-1, 0,1,-1, 1,-1,0], [0,0,0, 0,0,0, 0,0,1]],
    [[0,0,0, 0,-1,-1, 0,0,1], [0,0,0, 1,0,0, 0,0,0]],

    # counter strategy
    [[0,0,0, 0,0,0, -1,0,0], [0,0,0, 0,1,0, 0,0,0]],
    [[0,0,-1, 0,0,0, 0,0,0], [0,0,0, 0,1,0, 0,0,0]],
    [[-1,0,0, 0,0,0, 0,0,0], [0,0,0, 0,1,0, 0,0,0]],
    [[0,0,0, 0,0,0, 0,0,-1], [0,0,0, 0,1,0, 0,0,0]],

    [[-1,0,0, 0,1,0, 0,0,-1], [0,1,0, 1,0,1, 0,1,0]],
    [[0,0,-1, 0,1,0, -1,0,0], [0,1,0, 1,0,1, 0,1,0]],

    [[-1,1,0, 0,1,0, 0,-1,-1], [0,0,0, 0,0,0, 1,0,0]],
    [[-1,0,0, 1,1,-1, 0,0,-1], [0,0,1, 0,0,0, 0,0,0]],
    [[-1,0,0, -1,1,1, 0,0,-1], [0,0,0, 0,0,0, 1,0,0]],
    [[-1,-1,0, 0,1,0, 0,1,-1], [0,0,1, 0,0,0, 0,0,0]],

    [[-1,1,-1, 0,1,0, 1,-1,-1], [0,0,0, 0,0,1, 0,0,0]],
    [[-1,0,1, 1,1,-1, -1,0,-1], [0,0,0, 0,0,0, 0,1,0]],
    [[-1,0,-1, -1,1,1, 1,0,-1], [0,1,0, 0,0,0, 0,0,0]],
    [[-1,-1,1, 0,1,0, -1,1,-1], [0,0,0, 1,0,0, 0,0,0]],
])

# comment to get 98.51%
# fixes = mutate_data(fixes)
fixes = np.concatenate([fixes] * 150)

# merge data
data = np.concatenate([ data, favs, fixes ])
np.random.shuffle(data)

# get train data
train_x, train_y = data[:, 0], data[:, 1]

# validation data
np.random.shuffle(data)
nval = math.ceil(data.shape[0] * .1)
test_x, test_y = data[:nval, 0], data[:nval, 1]

In [133]:
# build neural network
nn = Sequential()
nn.add(Dense(9, input_dim=9))
nn.add(Dense(27, activation="tanh"))
nn.add(Dense(81, activation="tanh"))
nn.add(Dropout(.1))
nn.add(Dense(27, activation="tanh"))
nn.add(Dense(9, activation="sigmoid"))

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_139 (Dense)            (None, 9)                 90        
_________________________________________________________________
dense_140 (Dense)            (None, 27)                270       
_________________________________________________________________
dense_141 (Dense)            (None, 81)                2268      
_________________________________________________________________
dropout_19 (Dropout)         (None, 81)                0         
_________________________________________________________________
dense_142 (Dense)            (None, 27)                2214      
_________________________________________________________________
dense_143 (Dense)            (None, 9)                 252       
Total params: 5,094
Trainable params: 5,094
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# checkpoint and early stop
callbacks = [
    ModelCheckpoint("weights.best.hdf5", monitor="val_acc", verbose=1, save_best_only=True, mode="max"),
    EarlyStopping(monitor="val_acc", patience=50, mode="max"),
]

# train network
nn.fit(train_x, train_y, epochs=500, batch_size=32, validation_data=(test_x, test_y), callbacks=callbacks)

# evaluate the network
scores = nn.evaluate(train_x, train_y)
print("\n%s: %.2f%%" % (nn.metrics_names[1], scores[1]*100))

Train on 29583 samples, validate on 2959 samples
Epoch 1/500
29583/29583 [==============================] - 10s 326us/step - loss: 0.3759 - acc: 0.8595 - val_loss: 0.3230 - val_acc: 0.8716

Epoch 00001: val_acc improved from -inf to 0.87158, saving model to weights.best.hdf5
Epoch 2/500
29583/29583 [==============================] - 8s 271us/step - loss: 0.2979 - acc: 0.8830 - val_loss: 0.2777 - val_acc: 0.8879

Epoch 00002: val_acc improved from 0.87158 to 0.88791, saving model to weights.best.hdf5
Epoch 3/500
29583/29583 [==============================] - 9s 288us/step - loss: 0.2652 - acc: 0.8959 - val_loss: 0.2521 - val_acc: 0.9017

Epoch 00003: val_acc improved from 0.88791 to 0.90173, saving model to weights.best.hdf5
Epoch 4/500
29583/29583 [==============================] - 8s 287us/step - loss: 0.2452 - acc: 0.9032 - val_loss: 0.2328 - val_acc: 0.9081

Epoch 00004: val_acc improved from 0.90173 to 0.90811, saving model to weights.best.hdf5
Epoch 5/500
29583/29583 [============

29583/29583 [==============================] - 7s 230us/step - loss: 0.1119 - acc: 0.9561 - val_loss: 0.0912 - val_acc: 0.9630

Epoch 00037: val_acc did not improve from 0.96346
Epoch 38/500
29583/29583 [==============================] - 7s 230us/step - loss: 0.1108 - acc: 0.9567 - val_loss: 0.0907 - val_acc: 0.9640

Epoch 00038: val_acc improved from 0.96346 to 0.96403, saving model to weights.best.hdf5
Epoch 39/500
29583/29583 [==============================] - 7s 236us/step - loss: 0.1097 - acc: 0.9573 - val_loss: 0.0895 - val_acc: 0.9649

Epoch 00039: val_acc improved from 0.96403 to 0.96489, saving model to weights.best.hdf5
Epoch 40/500
29583/29583 [==============================] - 7s 232us/step - loss: 0.1087 - acc: 0.9575 - val_loss: 0.0878 - val_acc: 0.9649

Epoch 00040: val_acc improved from 0.96489 to 0.96493, saving model to weights.best.hdf5
Epoch 41/500
29583/29583 [==============================] - 7s 227us/step - loss: 0.1080 - acc: 0.9581 - val_loss: 0.0873 - val_acc:

In [131]:
board = np.array([
    0,  0, 0,
    0,  -1, 0,
   1,  0, 0
])
print(suggest(nn, board))

(0, 2)


In [ ]:
nn.save("model2.h5")